In [1]:
# установка абсолютного пути к проекту
# не нужно в случае `pip install`
%run -i ../tools/setup_env.py

## Примеры использования `torchcnnbuilder.builder`

## Обратите внимание, что данный пример содержит демонстрацию кастомных ошибок, реализованных в библиотеке, которые пользователь может получить при работе с комопонентами класса-строителя

Этот подмодуль содержит класс `Builder`, который создает последовательности свёрточных и транспонированных свёрточных слоев

### `Builder`

In [2]:
from torchcnnbuilder.builder import Builder

Параметры инициализации:

- **input_size** (Optional[Sequence[int]]): размер входного тензора. По умолчанию: None
- **minimum_feature_map_size** (Sequence[int] | int): минимальный размер карты признаков. По умолчанию: 5
- **max_channels** (int): максимальное количество каналов после любой свёртки. По умолчанию: 512
- **min_channels** (int): минимальное количество каналов после любой свёртки. По умолчанию: 32
- **activation_function** (nn.Module): функция активации. По умолчанию: nn.ReLU(inplace=True)
- **finish_activation_function** (Optional[nn.Module] | str): последняя функция активации, может быть такой же как activation_function (строка `'same'`). По умолчанию: None

Другие атрибуты:
- **conv_channels** (List[int]): список выходных каналов после каждого свёрточного слоя
- **transpose_conv_channels** (List[int]): список выходных каналов после каждого транспонированного свёрточного слоя
- **conv_layers** (List[tuple]): список размеров выходных тензоров после каждого свёрточного слоя
- **transpose_conv_layers** (List[tuple]): список размеров выходных тензоров после каждого транспонированного свёрточного слоя

Предположим, мы создаем CNN-модель для тензора размером 100 на 100, пусть наименьшая карта признаков после свёрток будет размером 3 на 3

In [3]:
builder = Builder(input_size=[125, 125], 
                  minimum_feature_map_size=3)

#### `Builder.build_convolve_sequence`

Параметры:

- **n_layers**: количество свёрточных слоев в части энкодера
- **in_channels**: количество каналов в первом входном тензоре. По умолчанию: 1
- **params**: параметры свёрточного слоя (`nn.Conv2d`). По умолчанию: None
- **normalization**: выбор нормализации между строками `dropout`, `instancenorm` и `batchnorm`. По умолчанию: None
- **sub_blocks**: количество свёрток в одном слое. По умолчанию: 1
- **p**: вероятность обнуления элемента (для `dropout`). По умолчанию: 0.5
- **inplace**: если установлено в True, будет выполнять операцию на месте (для `dropout`). По умолчанию: False
- **eps**: значение, добавляемое в знаменатель для численной стабильности (для `batchnorm`/`instancenorm`). По умолчанию: 1e-5
- **momentum**: используется для вычисления running_mean или running_var. Может быть None для кумулятивного скользящего среднего (для `batchnorm`/`instancenorm`). По умолчанию: 0.1
- **affine**: логическое значение, которое при установке в True делает этот модуль имеющим обучаемые аффинные параметры (для `batchnorm`/`instancenorm`). По умолчанию: True
- **ratio**: множитель для геометрической прогрессии увеличения каналов (карт признаков). Используется для `channel_growth_rate` как `exponential` или `power`. По умолчанию: 2 (степени двойки)
- **start**: начальная позиция геометрической прогрессии в случае `channel_growth_rate=exponential`. По умолчанию: 32
- **channel_growth_rate**: способ вычисления количества карт признаков между `exponential`, `proportion`, `power`, `linear` и `constant`. По умолчанию: `exponential`
- **conv_dim**: размерность свёрточной операции. По умолчанию: 2

Возвращает: nn.Sequential последовательность свёрток

> ПРЕДУПРЕЖДЕНИЕ: Вы не можете использовать этот метод без параметра `input_size` при инициализации `Builder`

Метод помогает строить свёрточные последовательности без написания всего кода вручную. Функция активации - это та, которая была задана во время инициализации всего класса. Возможно использование различных типов нормализации, но для манипуляции гиперпараметрами этого слоя необходимо строить свёрточную последовательность из свёрточных блоков отдельно *(метод `build_convolve_block`)*. Количество каналов (карт признаков) после каждого слоя вычисляется в зависимости от параметра `channel_growth_rate`:

$$
\begin{cases} 
    channel_i = start \times ratio^{i}, \quad i={1...n} & \text{если } \text{exponential} \\
    stop = \lfloor \displaystyle\frac{(input\_size[0] + input\_size[1]) * 0.5}{2} \rfloor + in\_channels & \text{если} \text{proportion} \\
    step = \displaystyle\frac{stop - in\_channels}{n\_layers} \\ 
    channels = \text{range}(in\_channels, stop, step) \\
    channel_i = in\_channels + i, \quad i={1...n}  & \text{если } \text{linear} \\
    channel_i = (in\_channels + i)^{ratio} \quad i={1...n} & \text{если} \text{power} \\
    channel_i = in\_channels, \quad i={1...n}  & \text{если} \text{constant}
\end{cases}
$$


In [4]:
builder.build_convolve_sequence(n_layers=3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
)

Иногда выбранные вами параметры свёрточного слоя могут привести к вырождению тензора после одного из слоев или его размер станет меньше указанного минимального размера карты признаков. В этом случае вы получите соответствующую ошибку

In [5]:
builder.build_convolve_sequence(n_layers=5, 
                                params={'kernel_size': 25, 'padding': 1, 'dilation': 2})

ValueError: Input size and parameters can not provide more than 4 layers.

Слои нормализации идут со стандартными параметрами как в `torch`. Для изменения параметров свёрточных слоев передайте их в виде словаря

In [6]:
builder.build_convolve_sequence(n_layers=2, 
                                in_channels=3,
                                params={'kernel_size': 5, 'padding': 1})

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
)

In [7]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                normalization='batchnorm',
                                eps=1e-3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [8]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                normalization='dropout',
                                p=0.1)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.1, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.1, inplace=False)
    (2): ReLU(inplace=True)
  )
)

Вы можете построить свёрточную последовательность из свёрточных блоков. Каждый свёрточный блок представляет собой свёрточные слои один за другим, проходя через которые тензор не изменяет свой размер. Хотя это ранний функционал, нет смысла делать свёрточную последовательность из таких блоков, поскольку тензор не изменит свой размер после прохождения через всю последовательность, для этого необходимо добавить реализацию слоев пулинга

In [9]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                sub_blocks=2,
                                normalization='instancenorm')

Sequential(
  (conv 1): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
  )
  (conv 2): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(i

Пример использования вычисления каналов с помощью `channel_growth_rate=proportion`

In [10]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                params={'kernel_size': 9},
                                sub_blocks=2, 
                                channel_growth_rate='proportion')

Sequential(
  (conv 1): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(3, 34, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(34, 34, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
  )
  (conv 2): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(34, 65, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(65, 65, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
  )
)

Чтобы предотвратить случайное создание тяжёлых моделей, по умолчанию существует определённое ограничение на количество выходных каналов (карт признаков) после свёрточного слоя. Вы можете установить его при инициализации класса (или изменить позже как атрибут класса)

In [11]:
builder.build_convolve_sequence(n_layers=5,
                                params={'kernel_size': 9},
                                in_channels=3,
                                channel_growth_rate='exponential', # по умолчанию
                                ratio=3)

ValueError: There is too many channels [[864]]. Max channels 512 [layer 4].

Если вы уменьшите количество слоев в предыдущем случае, то ошибки не будет

In [12]:
builder.build_convolve_sequence(n_layers=3,
                                in_channels=3,
                                channel_growth_rate='power',
                                ratio=3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(64, 125, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv2d(125, 216, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
)

Также вы можете делать то же самое, используя 1d или 3d свёрточные слои, однако `Builder` должен получить одинаковое количество измерений в `input_size` *(или не менее чем `len(input_size) - conv_dim`)*. Все слои нормализации будут заменены соответствующими N-мерными методами

In [13]:
test_builder = Builder(input_size=(31,))
test_builder.build_convolve_sequence(n_layers=5,
                                    in_channels=1,
                                    normalization='dropout',
                                    conv_dim=1)

Sequential(
  (conv 1): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 4): Sequential(
    (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 5): Sequential(
    (0): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
)

In [14]:
error_test_builder = Builder(input_size=(31,))
error_test_builder.build_convolve_sequence(n_layers=5,
                                           in_channels=1,
                                           normalization='instancenorm',
                                           conv_dim=3)

ValueError: The difference in dimensions between input_size input_size=(31,) and convolution conv_dim=3 should not be more than 1 (input_size.shape - conv_dim should be equal to 1 or 0).

#### `Builder.build_convolve_block`

Параметры:

- **in_channels**: количество каналов во входном изображении
- **out_channels**: количество каналов, производимых свёрткой
- **params**: параметры свёрточного слоя (`nn.Conv2d`). По умолчанию: None
- **normalization**: выбор нормализации между строками `dropout`, `instancenorm` и `batchnorm`. По умолчанию: None
- **sub_blocks**: количество свёрток в одном слое. По умолчанию: 1
- **p**: вероятность обнуления элемента (для `dropout`). По умолчанию: 0.5
- **inplace**: если установлено в True, будет выполнять операцию на месте (для `dropout`). По умолчанию: False
- **eps**: значение, добавляемое в знаменатель для численной стабильности (для `batchnorm`/`instancenorm`). По умолчанию: 1e-5
- **momentum**: используется для вычисления running_mean или running_var. Может быть None для кумулятивного скользящего среднего (для `batchnorm`/`instancenorm`). По умолчанию: 0.1
- **affine**: логическое значение, которое при установке в True делает этот модуль имеющим обучаемые аффинные параметры (для `batchnorm`/`instancenorm`). По умолчанию: True
- **conv_dim**: размерность свёрточной операции. По умолчанию: 2

Возвращает: nn.Sequential один свёрточный блок с функцией активации

Если вы хотите более тонкий выбор гиперпараметров свёрточных слоев, лучше использовать логику свёрточных блоков. В каждом таком блоке вы можете настроить свёртки и слои нормализации *(по умолчанию все параметры как в `torch`)*, а затем объединить эти блоки в `nn.Sequential`

In [15]:
import torch
input_image = torch.rand(1, 3, 100, 100)

In [16]:
conv_layer = builder.build_convolve_block(in_channels=3, 
                                          out_channels=64, 
                                          normalization='dropout',
                                          p=0.2,
                                          sub_blocks=3)
conv_layer

Sequential(
  (sub-block 1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (sub-block 2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (sub-block 3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
)

In [17]:
# проверка того, что тензор проходит через блок
output = conv_layer(input_image)
output.shape

torch.Size([1, 64, 100, 100])

In [18]:
conv_layer = builder.build_convolve_block(in_channels=3, 
                                          out_channels=64, 
                                          params={'kernel_size': (7, 7), 'dilation': (3, 3)},
                                          normalization='instancenorm',
                                          eps=1e-7)
conv_layer

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), dilation=(3, 3))
  (1): InstanceNorm2d(64, eps=1e-07, momentum=0.1, affine=True, track_running_stats=False)
  (2): ReLU(inplace=True)
)

In [19]:
# проверка того, что тензор проходит через блок
output = conv_layer(input_image)
output.shape

torch.Size([1, 64, 82, 82])

Вы также можете изменить размерность слоев блока, используя `conv_dim`

In [20]:
builder.build_convolve_block(in_channels=3,
                             out_channels=4,
                             normalization='dropout',
                             conv_dim=3)

Sequential(
  (0): Conv3d(3, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (1): Dropout3d(p=0.5, inplace=False)
  (2): ReLU(inplace=True)
)

#### `Builder.build_transpose_convolve_sequence`

Параметры: 

- **n_layers**: количество свёрточных слоев в части энкодера
- **in_channels**: количество каналов в первом входном тензоре. По умолчанию: 1
- **out_channels**: количество каналов после последовательности транспонированных свёрток. По умолчанию: 1
- **out_size**: выходной размер после последовательности транспонированных свёрток. По умолчанию: None (размер входа)
- **params**: параметры свёрточного слоя (`nn.Conv2d`). По умолчанию: None
- **normalization**: выбор нормализации между строками `dropout`, `instancenorm` и `batchnorm`. По умолчанию: None
- **sub_blocks**: количество свёрток в одном слое. По умолчанию: 1
- **p**: вероятность обнуления элемента (для `dropout`). По умолчанию: 0.5
- **inplace**: если установлено в True, будет выполнять операцию на месте (для `dropout`). По умолчанию: False
- **eps**: значение, добавляемое в знаменатель для численной стабильности (для `batchnorm`/`instancenorm`). По умолчанию: 1e-5
- **momentum**: используется для вычисления running_mean или running_var. Может быть None для кумулятивного скользящего среднего (для `batchnorm`/`instancenorm`). По умолчанию: 0.1
- **affine**: логическое значение, которое при установке в True делает этот модуль имеющим обучаемые аффинные параметры (для `batchnorm`/`instancenorm`). По умолчанию: True
- **ratio**: множитель для геометрической прогрессии увеличения каналов (карт признаков). Используется для `channel_growth_rate` как `exponential` или `power`. По умолчанию: 2 (степени двойки)
- **start**: начальная позиция геометрической прогрессии в случае `channel_growth_rate=exponential`. По умолчанию: 32
- **channel_growth_rate**: способ вычисления количества карт признаков между `exponential`, `proportion`, `power`, `linear` и `constant`. По умолчанию: `exponential`
- **conv_dim**: размерность свёрточной операции. По умолчанию: 2
- **adaptive_pool**: выбор последнего слоя как адаптивного пулинга между строками `avgpool` или `maxpool`. По умолчанию: `avgpool`

Возвращает: nn.Sequential последовательность транспонированных свёрток

Метод помогает строить последовательности транспонированных свёрток без написания всего кода вручную. Функция активации - это та, которая была задана во время инициализации всего класса. Возможно использование различных типов нормализации, но для манипуляции гиперпараметрами этого слоя необходимо строить последовательность транспонированных свёрток из блоков транспонированных свёрток отдельно *(метод `build_transpose_convolve_block`)*. Количество каналов (карт признаков) после каждого слоя вычисляется в зависимости от параметра `channel_growth_rate`. См. формулы вычисления выше в функции, здесь всё рассчитывается так же, но в обратном направлении. Если функция активации была указана при инициализации класса *(см. атрибуты при инициализации)* после последнего слоя, то она будет применена

Для устранения проблемы несоответствия размеров между выходным тензором после части декодера и входным тензором перед частью энкодера *(из-за чётности/нечётности параметров свёртки некоторые пиксели могут быть потеряны)*, в конце есть слой пулинга

> ПРЕДУПРЕЖДЕНИЕ: Вы не можете использовать этот метод без параметра `input_size` при инициализации `Builder`

In [21]:
builder.build_transpose_convolve_sequence(n_layers=3)

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(216, 108, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(108, 54, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(54, 1, kernel_size=(3, 3), stride=(1, 1))
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [22]:
builder.build_transpose_convolve_sequence(n_layers=2, 
                                          in_channels=3,
                                          params={'kernel_size': 5, 'padding': 1})

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(3, 1, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(1, 1, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [23]:
builder.build_transpose_convolve_sequence(n_layers=2,
                                          in_channels=10,
                                          out_channels=3,
                                          out_size=(130, 130),
                                          normalization='dropout',
                                          p=0.3,
                                          channel_growth_rate='linear')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(10, 9, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(9, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
  )
  (resize): AdaptiveAvgPool2d(output_size=(130, 130))
)

Иногда выбранные вами параметры транспонированного свёрточного слоя могут привести к вырождению количества карт признаков после одного из слоев или оно станет меньше указанного минимума каналов. В этом случае вы получите соответствующую ошибку

In [24]:
builder.build_transpose_convolve_sequence(n_layers=6,
                                          in_channels=20,
                                          ratio=3,
                                          channel_growth_rate='exponential') # по умолчанию

C:\Users\Julia\Documents\NSS_lab\TorchCNNBuilder\venv\lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


ValueError: There is too few channels [0]. You can not provide less then 1 channel [layer 3].

Примеры различных финальных функций активации

In [25]:
import torch.nn as nn

# переопределение последней функции активации
builder.finish_activation_function = nn.Softmax()

builder.build_transpose_convolve_sequence(n_layers=3,
                                          in_channels=20,
                                          out_channels=3,
                                          normalization='dropout')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(20, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(10, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(5, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): Softmax(dim=None)
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [26]:
# переопределение последней функции активации
builder.finish_activation_function = 'same'

builder.build_transpose_convolve_sequence(n_layers=3,
                                          in_channels=30,
                                          out_channels=2,
                                          normalization='batchnorm',
                                          adaptive_pool='maxpool')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(30, 15, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(15, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(7, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (resize): AdaptiveMaxPool2d(output_size=(125, 125))
)

#### `Builder.build_transpose_convolve_block`

Параметры:

- **in_channels**: количество каналов во входном изображении
- **out_channels**: количество каналов, производимых свёрткой
- **params**: параметры свёрточного слоя (nn.Conv2d). По умолчанию: None
- **normalization**: выбор нормализации между строками `dropout`, `instancenorm` и `batchnorm`. По умолчанию: None
- **sub_blocks**: количество свёрток в одном слое. По умолчанию: 1
- **p**: вероятность обнуления элемента. По умолчанию (для `dropout`): 0.5
- **inplace**: если установлено в True, будет выполнять операцию на месте. По умолчанию (для `dropout`): False
- **eps**: значение, добавляемое в знаменатель для численной стабильности (для `batchnorm`/`instancenorm`). По умолчанию: 1e-5
- **momentum**: используется для вычисления running_mean или running_var. Может быть None для кумулятивного скользящего среднего (для `batchnorm`/`instancenorm`). По умолчанию: 0.1
- **affine**: логическое значение, которое при установке в True делает этот модуль имеющим обучаемые аффинные параметры (для `batchnorm`/`instancenorm`). По умолчанию: True
- **last_block**: если True, отсутствует функция активации после транспонированной свёртки. По умолчанию: False
- **conv_dim**: размерность свёрточной операции. По умолчанию: 2

Возвращает: nn.Sequential один свёрточный блок с функцией активации

Если вы хотите более тонкий выбор гиперпараметров транспонированных свёрточных слоев, лучше использовать логику блоков транспонированных свёрток. В каждом таком блоке вы можете настроить транспонированные свёртки и слои нормализации *(по умолчанию все параметры как в `torch`)*, а затем объединить эти блоки в `nn.Sequential`

In [27]:
builder.finish_activation_function = nn.Softmax()
deconv_layer = builder.build_transpose_convolve_block(in_channels=3, 
                                                      out_channels=64, 
                                                      normalization='dropout',
                                                      p=0.2,
                                                      sub_blocks=3,
                                                      last_block=True)
deconv_layer

Sequential(
  (transpose sub-block 1): Sequential(
    (0): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (transpose sub-block 2): Sequential(
    (0): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (transpose sub-block 3): Sequential(
    (0): ConvTranspose2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): Softmax(dim=None)
  )
)

In [28]:
deconv_layer = builder.build_transpose_convolve_block(in_channels=64, 
                                                      out_channels=3,
                                                      normalization='batchnorm',
                                                      eps=1e-4)
deconv_layer

Sequential(
  (0): ConvTranspose2d(64, 3, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(3, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)

#### Примеры атрибутов и сравнение блоков

Давайте сравним свёрточный блок и транспонированный свёрточный блок

In [29]:
conv_layer = builder.build_convolve_block(in_channels=3,
                                          out_channels=64,
                                          normalization='batchnorm',
                                          eps=1e-4)
conv_layer

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)

In [30]:
input_image = torch.rand(1, 3, 125, 125)

In [31]:
after_conv = conv_layer(input_image)
after_conv.shape

torch.Size([1, 64, 123, 123])

In [32]:
after_deconv = deconv_layer(after_conv)
after_deconv.shape

torch.Size([1, 3, 125, 125])

Как мы видим, размеры входного и финального выходного тензора одинаковы. Более того, вы можете проверить историю размеров тензоров после последовательностей свёрток и транспонированных свёрток (как историю каналов)

In [33]:
print(f'История каналов свёрток: {builder.conv_channels}',
      f'История каналов транспонированных свёрток: {builder.transpose_conv_channels}',
      f'История размеров свёрток: {builder.conv_layers}',
      f'История размеров транспонированных свёрток: {builder.transpose_conv_layers}',
      sep='\n')

История каналов свёрток: [3, 64, 125, 216]
История каналов транспонированных свёрток: [30, 15, 7, 2]
История размеров свёрток: [(125, 125), (123, 123), (121, 121), (119, 119)]
История размеров транспонированных свёрток: [(119, 119), (121, 121), (123, 123), (125, 125)]
